# ARCO Example: Quick Pattern Analysis

This notebook demonstrates how to use ARCO for quick XRD pattern analysis.

**What you'll learn:**
- Load XRD data from CSV
- Compute ARCO features in 2 lines of code
- Interpret RCI and top rationals
- Visualize results

In [ ]:
# Setup
import sys
sys.path.insert(0, '../src')  # Add src to path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nomad_auto_xrd.lib import compute_arco_features

print("✓ Imports successful")

## Load Sample Data

We'll use a synthetic XRD pattern with periodic peaks.

In [ ]:
# Load sample data
df = pd.read_csv('data/sample_xrd_pattern.csv')

two_theta = df['two_theta'].values
intensity = df['intensity'].values

print(f"Loaded {len(two_theta)} data points")
print(f"2θ range: {two_theta.min():.1f}° to {two_theta.max():.1f}°")
print(f"Intensity range: {intensity.min():.1f} to {intensity.max():.1f}")

## Visualize Raw Pattern

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(two_theta, intensity, linewidth=1)
plt.xlabel('2θ (degrees)')
plt.ylabel('Intensity (a.u.)')
plt.title('XRD Pattern')
plt.grid(True, alpha=0.3)
plt.show()

## Compute ARCO Features

**Just 2 lines of code!**

In [ ]:
# Compute ARCO features
features = compute_arco_features(
    two_theta=two_theta,
    intensity=intensity,
    Qmax=40,      # Max denominator for rational anchors
    alpha=0.5,    # Bandwidth scale
    major_q=20    # Major rationals threshold
)

print("✓ ARCO analysis complete")

## View Results

In [ ]:
# RCI (Rational Coherence Index)
print(f"RCI (Periodicity Score): {features['rci']:.4f}")
print(f"  → Range: 0 (no periodicity) to 1 (strong periodicity)")
print()

# ARCO fingerprint
print(f"ARCO Fingerprint: {len(features['arco_print'])} features")
print(f"  → Fixed-length vector for ML/similarity search")
print()

# Top rationals
print("Top 5 Rational Frequencies:")
for i, rational in enumerate(features['top_rationals'][:5], 1):
    freq = rational['frequency']
    power = rational['power']
    q = rational['denominator']
    period = 1/freq if freq > 0 else float('inf')
    print(f"  {i}. Frequency {freq:.4f} (1/{q}): power={power:.4f}, period≈{period:.1f}")

## Interpret Results

**RCI Interpretation:**
- **> 0.7**: Strong periodic structure (highly crystalline)
- **0.4-0.7**: Moderate periodicity (semi-crystalline)
- **< 0.4**: Weak periodicity (amorphous or random)

**Top Rationals:**
- Show dominant frequency components in the pattern
- Low denominators (q ≤ 10) indicate simple periodic structures
- High denominators suggest complex or long-range periodicities

## Visualize ARCO Fingerprint

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot full fingerprint
axes[0].plot(features['arco_print'], linewidth=0.5, alpha=0.8)
axes[0].set_xlabel('Feature Index')
axes[0].set_ylabel('Arc Power')
axes[0].set_title(f'ARCO Fingerprint (RCI={features["rci"]:.4f})')
axes[0].grid(True, alpha=0.3)

# Plot first 200 features (zoomed in)
axes[1].plot(features['arco_print'][:200], linewidth=0.8)
axes[1].set_xlabel('Feature Index')
axes[1].set_ylabel('Arc Power')
axes[1].set_title('ARCO Fingerprint (first 200 features, zoomed)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Try Your Own Data

Replace the sample data with your own XRD measurements:

In [ ]:
# Example with your data:
# my_two_theta = ... # load your 2θ values
# my_intensity = ... # load your intensity values
#
# my_features = compute_arco_features(
#     two_theta=my_two_theta,
#     intensity=my_intensity,
#     Qmax=40,
#     alpha=0.5
# )
#
# print(f"Your pattern RCI: {my_features['rci']:.4f}")

## Next Steps

1. **Tune parameters**: Adjust `Qmax`, `alpha`, `major_q` for your application
2. **Compare patterns**: Use ARCO fingerprints for similarity analysis
3. **Statistical validation**: Use z-scores to test significance
4. **Advanced features**: Try multi-track analysis, ARCO-3D, etc.

See `notebooks/arco_xrd_demo.ipynb` for comprehensive tutorial!